In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from env import user, password, host
from scipy.stats import levene, ttest_ind
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
import math
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import statsmodels.api as sm
import wrangle as w
import warnings
warnings.filterwarnings("ignore")

In [2]:
df22 = pd.read_csv('CAMPUS_summary_22.csv')

In [3]:
df22

,AGGREGATION LEVEL,CAMPUS,REGION,DISTRICT NAME AND NUMBER,CHARTER_STATUS,CAMPUS NAME AND NUMBER,SECTION,HEADING,HEADING NAME,YR22
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,533
1,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A02,CAMPUS DISCIPLINE POPULATION,27
2,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,40
3,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,B-DISCIPLINE DATA TRENDS,B13,STUDENTS SUSPENDED OUT OF SCHOOL,27
4,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,F-OUT OF SCHOOL SUSPENSIONS,C24,BLACK OR AFRICAN AMERICAN,-999
...,...,...,...,...,...,...,...,...,...,...
323988,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,V-AT RISK IN SCHOOL SUS.,F19,NON AT RISK IN SCHOOL SUSPENSIONS,-999
323989,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,W-REASON INCIDENT COUNTS,G21,21-VIOLATED LOCAL CODE OF CONDUCT,-999
323990,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,W-REASON INCIDENT COUNTS,G61,61-BULLYING  TEC 37.0052(B),-999
323991,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,X-DISCIPLINE ACTION COUNTS,H05,05-OUT-OF-SCHOOL SUSPENSION,-999


In [4]:
df22=df22.rename(columns=
                     {'AGGREGATION LEVEL':'agg_level','CAMPUS':'campus_number','REGION':'region',
                      'DISTRICT NAME AND NUMBER':'dist_name_num','CHARTER_STATUS':'charter_status',
                      'CAMPUS NAME AND NUMBER': 'campus_name_num', 'SECTION': 'section',
                      'HEADING':'heading','HEADING NAME': 'heading_name','YR22':'student_count'})

In [5]:
df22

,agg_level,campus_number,region,dist_name_num,charter_status,campus_name_num,section,heading,heading_name,student_count
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,533
1,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A02,CAMPUS DISCIPLINE POPULATION,27
2,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,40
3,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,B-DISCIPLINE DATA TRENDS,B13,STUDENTS SUSPENDED OUT OF SCHOOL,27
4,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,F-OUT OF SCHOOL SUSPENSIONS,C24,BLACK OR AFRICAN AMERICAN,-999
...,...,...,...,...,...,...,...,...,...,...
323988,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,V-AT RISK IN SCHOOL SUS.,F19,NON AT RISK IN SCHOOL SUSPENSIONS,-999
323989,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,W-REASON INCIDENT COUNTS,G21,21-VIOLATED LOCAL CODE OF CONDUCT,-999
323990,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,W-REASON INCIDENT COUNTS,G61,61-BULLYING  TEC 37.0052(B),-999
323991,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,X-DISCIPLINE ACTION COUNTS,H05,05-OUT-OF-SCHOOL SUSPENSION,-999


In [6]:
df22['charter_encoded'] = df22.charter_status.map({'OPEN ENROLLMENT CHARTER': 1, 
                                                       'TRADITIONAL ISD/CSD':0})

In [7]:
df22

,agg_level,campus_number,region,dist_name_num,charter_status,campus_name_num,section,heading,heading_name,student_count,charter_encoded
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,533,1
1,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A02,CAMPUS DISCIPLINE POPULATION,27,1
2,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,40,1
3,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,B-DISCIPLINE DATA TRENDS,B13,STUDENTS SUSPENDED OUT OF SCHOOL,27,1
4,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,F-OUT OF SCHOOL SUSPENSIONS,C24,BLACK OR AFRICAN AMERICAN,-999,1
...,...,...,...,...,...,...,...,...,...,...,...
323988,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,V-AT RISK IN SCHOOL SUS.,F19,NON AT RISK IN SCHOOL SUSPENSIONS,-999,0
323989,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,W-REASON INCIDENT COUNTS,G21,21-VIOLATED LOCAL CODE OF CONDUCT,-999,0
323990,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,W-REASON INCIDENT COUNTS,G61,61-BULLYING  TEC 37.0052(B),-999,0
323991,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,X-DISCIPLINE ACTION COUNTS,H05,05-OUT-OF-SCHOOL SUSPENSION,-999,0


In [8]:
df22=df22[(df22.heading == 'A01') | (df22.heading ==  'A03')]

In [9]:
df22

,agg_level,campus_number,region,dist_name_num,charter_status,campus_name_num,section,heading,heading_name,student_count,charter_encoded
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,533,1
2,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,40,1
17,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,642,1
19,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,208,1
50,CAMPUS SUMMARY,57829001,10,A+ ACADEMY 057829,OPEN ENROLLMENT CHARTER,A+ ACADEMY EL 057829001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,905,1
...,...,...,...,...,...,...,...,...,...,...,...
323917,CAMPUS SUMMARY,3906101,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA EL 003906101,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,-999,0
323929,CAMPUS SUMMARY,3906001,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA H S 003906001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,160,0
323931,CAMPUS SUMMARY,3906001,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA H S 003906001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,-999,0
323965,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,224,0


In [10]:
df22.dropna()

,agg_level,campus_number,region,dist_name_num,charter_status,campus_name_num,section,heading,heading_name,student_count,charter_encoded
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,533,1
2,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,40,1
17,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,642,1
19,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,208,1
50,CAMPUS SUMMARY,57829001,10,A+ ACADEMY 057829,OPEN ENROLLMENT CHARTER,A+ ACADEMY EL 057829001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,905,1
...,...,...,...,...,...,...,...,...,...,...,...
323917,CAMPUS SUMMARY,3906101,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA EL 003906101,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,-999,0
323929,CAMPUS SUMMARY,3906001,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA H S 003906001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,160,0
323931,CAMPUS SUMMARY,3906001,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA H S 003906001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,-999,0
323965,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,224,0


In [11]:
df22=df22.drop_duplicates()

In [12]:
df22

,agg_level,campus_number,region,dist_name_num,charter_status,campus_name_num,section,heading,heading_name,student_count,charter_encoded
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,533,1
2,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,40,1
17,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,642,1
19,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,208,1
50,CAMPUS SUMMARY,57829001,10,A+ ACADEMY 057829,OPEN ENROLLMENT CHARTER,A+ ACADEMY EL 057829001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,905,1
...,...,...,...,...,...,...,...,...,...,...,...
323917,CAMPUS SUMMARY,3906101,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA EL 003906101,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,-999,0
323929,CAMPUS SUMMARY,3906001,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA H S 003906001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,160,0
323931,CAMPUS SUMMARY,3906001,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA H S 003906001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,-999,0
323965,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,224,0


In [13]:
df22=df22[df22['student_count'] != '-999']

In [14]:
df22

,agg_level,campus_number,region,dist_name_num,charter_status,campus_name_num,section,heading,heading_name,student_count,charter_encoded
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,533,1
2,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,40,1
17,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,642,1
19,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,208,1
50,CAMPUS SUMMARY,57829001,10,A+ ACADEMY 057829,OPEN ENROLLMENT CHARTER,A+ ACADEMY EL 057829001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,905,1
...,...,...,...,...,...,...,...,...,...,...,...
323891,CAMPUS SUMMARY,253901106,1,ZAPATA COUNTY ISD 253901,TRADITIONAL ISD/CSD,ZAPATA SOUTH EL 253901106,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,13,0
323915,CAMPUS SUMMARY,3906101,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA EL 003906101,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,178,0
323929,CAMPUS SUMMARY,3906001,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA H S 003906001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,160,0
323965,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,224,0


In [15]:
df22['student_count']= df22['student_count'].str.replace("<", "")

In [16]:
df22

,agg_level,campus_number,region,dist_name_num,charter_status,campus_name_num,section,heading,heading_name,student_count,charter_encoded
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,533,1
2,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,40,1
17,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,642,1
19,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,208,1
50,CAMPUS SUMMARY,57829001,10,A+ ACADEMY 057829,OPEN ENROLLMENT CHARTER,A+ ACADEMY EL 057829001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,905,1
...,...,...,...,...,...,...,...,...,...,...,...
323891,CAMPUS SUMMARY,253901106,1,ZAPATA COUNTY ISD 253901,TRADITIONAL ISD/CSD,ZAPATA SOUTH EL 253901106,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,13,0
323915,CAMPUS SUMMARY,3906101,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA EL 003906101,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,178,0
323929,CAMPUS SUMMARY,3906001,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA H S 003906001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,160,0
323965,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,224,0


In [17]:
df22['student_count'] = df22['student_count'].astype(float)

In [18]:
df22

,agg_level,campus_number,region,dist_name_num,charter_status,campus_name_num,section,heading,heading_name,student_count,charter_encoded
0,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,533.0,1
2,CAMPUS SUMMARY,57816102,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,GENESIS CAMPUS PRE-K THRU 5 057816102,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,40.0,1
17,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,642.0,1
19,CAMPUS SUMMARY,57816101,10,A W BROWN LEADERSHIP ACADEMY 057816,OPEN ENROLLMENT CHARTER,QUEST CAMPUS PRE-K THRU 8 057816101,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,208.0,1
50,CAMPUS SUMMARY,57829001,10,A+ ACADEMY 057829,OPEN ENROLLMENT CHARTER,A+ ACADEMY EL 057829001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,905.0,1
...,...,...,...,...,...,...,...,...,...,...,...
323891,CAMPUS SUMMARY,253901106,1,ZAPATA COUNTY ISD 253901,TRADITIONAL ISD/CSD,ZAPATA SOUTH EL 253901106,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,13.0,0
323915,CAMPUS SUMMARY,3906101,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA EL 003906101,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,178.0,0
323929,CAMPUS SUMMARY,3906001,7,ZAVALLA ISD 003906,TRADITIONAL ISD/CSD,ZAVALLA H S 003906001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,160.0,0
323965,CAMPUS SUMMARY,25906001,15,ZEPHYR ISD 025906,TRADITIONAL ISD/CSD,ZEPHYR SCHOOL 025906001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,224.0,0


In [19]:
dfpivot=df22.pivot(index='campus_number', columns='heading', values= 'student_count').dropna()

In [20]:
dfpivot

heading,A01,A03
campus_number,,
1902001,173.0,37.0
1903001,335.0,141.0
1903041,309.0,65.0
1903102,295.0,40.0
1904001,248.0,101.0
...,...,...
253901001,1028.0,239.0
253901041,856.0,298.0
253901106,516.0,13.0


In [21]:
df22=df22.merge(dfpivot,how= 'right', on= 'campus_number')

In [22]:
df22

,agg_level,campus_number,region,dist_name_num,charter_status,campus_name_num,section,heading,heading_name,student_count,charter_encoded,A01,A03
0,CAMPUS SUMMARY,1902001,7,CAYUGA ISD 001902,TRADITIONAL ISD/CSD,CAYUGA H S 001902001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,173.0,0,173.0,37.0
1,CAMPUS SUMMARY,1902001,7,CAYUGA ISD 001902,TRADITIONAL ISD/CSD,CAYUGA H S 001902001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,37.0,0,173.0,37.0
2,CAMPUS SUMMARY,1903001,7,ELKHART ISD 001903,TRADITIONAL ISD/CSD,ELKHART H S 001903001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,335.0,0,335.0,141.0
3,CAMPUS SUMMARY,1903001,7,ELKHART ISD 001903,TRADITIONAL ISD/CSD,ELKHART H S 001903001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,141.0,0,335.0,141.0
4,CAMPUS SUMMARY,1903041,7,ELKHART ISD 001903,TRADITIONAL ISD/CSD,ELKHART MIDDLE 001903041,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,309.0,0,309.0,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11863,CAMPUS SUMMARY,253901106,1,ZAPATA COUNTY ISD 253901,TRADITIONAL ISD/CSD,ZAPATA SOUTH EL 253901106,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,13.0,0,516.0,13.0
11864,CAMPUS SUMMARY,254901001,20,CRYSTAL CITY ISD 254901,TRADITIONAL ISD/CSD,CRYSTAL CITY H S 254901001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,556.0,0,556.0,310.0
11865,CAMPUS SUMMARY,254901001,20,CRYSTAL CITY ISD 254901,TRADITIONAL ISD/CSD,CRYSTAL CITY H S 254901001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,310.0,0,556.0,310.0
11866,CAMPUS SUMMARY,254901042,20,CRYSTAL CITY ISD 254901,TRADITIONAL ISD/CSD,STERLING H FLY JR H S 254901042,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,432.0,0,432.0,94.0


In [23]:
df22=df22.rename(columns={'A01': 'student_enrollment', 'A03':'discipline_count'})

In [25]:
df22

,agg_level,campus_number,region,dist_name_num,charter_status,campus_name_num,section,heading,heading_name,student_count,charter_encoded,student_enrollment,discipline_count
0,CAMPUS SUMMARY,1902001,7,CAYUGA ISD 001902,TRADITIONAL ISD/CSD,CAYUGA H S 001902001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,173.0,0,173.0,37.0
1,CAMPUS SUMMARY,1902001,7,CAYUGA ISD 001902,TRADITIONAL ISD/CSD,CAYUGA H S 001902001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,37.0,0,173.0,37.0
2,CAMPUS SUMMARY,1903001,7,ELKHART ISD 001903,TRADITIONAL ISD/CSD,ELKHART H S 001903001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,335.0,0,335.0,141.0
3,CAMPUS SUMMARY,1903001,7,ELKHART ISD 001903,TRADITIONAL ISD/CSD,ELKHART H S 001903001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,141.0,0,335.0,141.0
4,CAMPUS SUMMARY,1903041,7,ELKHART ISD 001903,TRADITIONAL ISD/CSD,ELKHART MIDDLE 001903041,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,309.0,0,309.0,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11863,CAMPUS SUMMARY,253901106,1,ZAPATA COUNTY ISD 253901,TRADITIONAL ISD/CSD,ZAPATA SOUTH EL 253901106,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,13.0,0,516.0,13.0
11864,CAMPUS SUMMARY,254901001,20,CRYSTAL CITY ISD 254901,TRADITIONAL ISD/CSD,CRYSTAL CITY H S 254901001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,556.0,0,556.0,310.0
11865,CAMPUS SUMMARY,254901001,20,CRYSTAL CITY ISD 254901,TRADITIONAL ISD/CSD,CRYSTAL CITY H S 254901001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,310.0,0,556.0,310.0
11866,CAMPUS SUMMARY,254901042,20,CRYSTAL CITY ISD 254901,TRADITIONAL ISD/CSD,STERLING H FLY JR H S 254901042,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,432.0,0,432.0,94.0


In [26]:
df22['discipline_percent']= ((df22['discipline_count']/df22['student_enrollment'])*100)

In [28]:
df22

,agg_level,campus_number,region,dist_name_num,charter_status,campus_name_num,section,heading,heading_name,student_count,charter_encoded,student_enrollment,discipline_count,discipline_percent
0,CAMPUS SUMMARY,1902001,7,CAYUGA ISD 001902,TRADITIONAL ISD/CSD,CAYUGA H S 001902001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,173.0,0,173.0,37.0,21.387283
1,CAMPUS SUMMARY,1902001,7,CAYUGA ISD 001902,TRADITIONAL ISD/CSD,CAYUGA H S 001902001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,37.0,0,173.0,37.0,21.387283
2,CAMPUS SUMMARY,1903001,7,ELKHART ISD 001903,TRADITIONAL ISD/CSD,ELKHART H S 001903001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,335.0,0,335.0,141.0,42.089552
3,CAMPUS SUMMARY,1903001,7,ELKHART ISD 001903,TRADITIONAL ISD/CSD,ELKHART H S 001903001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,141.0,0,335.0,141.0,42.089552
4,CAMPUS SUMMARY,1903041,7,ELKHART ISD 001903,TRADITIONAL ISD/CSD,ELKHART MIDDLE 001903041,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,309.0,0,309.0,65.0,21.035599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11863,CAMPUS SUMMARY,253901106,1,ZAPATA COUNTY ISD 253901,TRADITIONAL ISD/CSD,ZAPATA SOUTH EL 253901106,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,13.0,0,516.0,13.0,2.519380
11864,CAMPUS SUMMARY,254901001,20,CRYSTAL CITY ISD 254901,TRADITIONAL ISD/CSD,CRYSTAL CITY H S 254901001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,556.0,0,556.0,310.0,55.755396
11865,CAMPUS SUMMARY,254901001,20,CRYSTAL CITY ISD 254901,TRADITIONAL ISD/CSD,CRYSTAL CITY H S 254901001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,310.0,0,556.0,310.0,55.755396
11866,CAMPUS SUMMARY,254901042,20,CRYSTAL CITY ISD 254901,TRADITIONAL ISD/CSD,STERLING H FLY JR H S 254901042,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,432.0,0,432.0,94.0,21.759259


In [29]:
df22=df22.round({'discipline_percent': 0})

In [30]:
df22

,agg_level,campus_number,region,dist_name_num,charter_status,campus_name_num,section,heading,heading_name,student_count,charter_encoded,student_enrollment,discipline_count,discipline_percent
0,CAMPUS SUMMARY,1902001,7,CAYUGA ISD 001902,TRADITIONAL ISD/CSD,CAYUGA H S 001902001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,173.0,0,173.0,37.0,21.0
1,CAMPUS SUMMARY,1902001,7,CAYUGA ISD 001902,TRADITIONAL ISD/CSD,CAYUGA H S 001902001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,37.0,0,173.0,37.0,21.0
2,CAMPUS SUMMARY,1903001,7,ELKHART ISD 001903,TRADITIONAL ISD/CSD,ELKHART H S 001903001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,335.0,0,335.0,141.0,42.0
3,CAMPUS SUMMARY,1903001,7,ELKHART ISD 001903,TRADITIONAL ISD/CSD,ELKHART H S 001903001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,141.0,0,335.0,141.0,42.0
4,CAMPUS SUMMARY,1903041,7,ELKHART ISD 001903,TRADITIONAL ISD/CSD,ELKHART MIDDLE 001903041,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,309.0,0,309.0,65.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11863,CAMPUS SUMMARY,253901106,1,ZAPATA COUNTY ISD 253901,TRADITIONAL ISD/CSD,ZAPATA SOUTH EL 253901106,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,13.0,0,516.0,13.0,3.0
11864,CAMPUS SUMMARY,254901001,20,CRYSTAL CITY ISD 254901,TRADITIONAL ISD/CSD,CRYSTAL CITY H S 254901001,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,556.0,0,556.0,310.0,56.0
11865,CAMPUS SUMMARY,254901001,20,CRYSTAL CITY ISD 254901,TRADITIONAL ISD/CSD,CRYSTAL CITY H S 254901001,A-PARTICIPATION,A03,CAMPUS DISCIPLINE RECORD COUNT,310.0,0,556.0,310.0,56.0
11866,CAMPUS SUMMARY,254901042,20,CRYSTAL CITY ISD 254901,TRADITIONAL ISD/CSD,STERLING H FLY JR H S 254901042,A-PARTICIPATION,A01,CAMPUS CUMULATIVE YEAR END ENROLLMENT,432.0,0,432.0,94.0,22.0


In [31]:
df22=df22.drop(columns=['agg_level', 'campus_number', 'region', 'charter_status', 'dist_name_num','section', 'heading','heading_name', 'student_count'])

In [32]:
df22

,campus_name_num,charter_encoded,student_enrollment,discipline_count,discipline_percent
0,CAYUGA H S 001902001,0,173.0,37.0,21.0
1,CAYUGA H S 001902001,0,173.0,37.0,21.0
2,ELKHART H S 001903001,0,335.0,141.0,42.0
3,ELKHART H S 001903001,0,335.0,141.0,42.0
4,ELKHART MIDDLE 001903041,0,309.0,65.0,21.0
...,...,...,...,...,...
11863,ZAPATA SOUTH EL 253901106,0,516.0,13.0,3.0
11864,CRYSTAL CITY H S 254901001,0,556.0,310.0,56.0
11865,CRYSTAL CITY H S 254901001,0,556.0,310.0,56.0
11866,STERLING H FLY JR H S 254901042,0,432.0,94.0,22.0


In [ ]:
STOP

In [ ]:
def df_combine(a,b,c,d,e):
    df=pd.concat([df18,df19,df20,df21,df22], ignore_index=True)
    return(df)

#call function with: df_combine(df19,df20,df21,df22)

In [ ]:
df_combine(df18,df19,df20,df21,df22)

In [ ]:
train, validate, test= split_tea_data(df) 

In [ ]:
train.head()